In [ ]:
Ex_GooglePlay

In [ ]:
# 如何爬取 GooglePlay 的APP留言與評分
到 Google Play 的 APP 頁面抓取客戶的留言，如果再搭配一些文字分析的技巧，我們就能更快速、即時的了解 APP 的優缺點。
透過 Selenium 幫我們自動抓取Google Play 上 Foodpanda 的客戶評分與留言等資料

In [ ]:
抓取 Googleplay 的 APP 評論與評分
安裝套件

In [2]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd 
import time

In [ ]:
打開想爬的網頁
在此以 Foodpanda 的APP頁面作為範例

https://play.google.com/store/apps/details?id=com.global.foodpanda.android&hl=zh_TW&showAllReviews=true

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(options=options)
driver.get('https://play.google.com/store/apps/details?id=com.global.foodpanda.android&hl=zh_TW&showAllReviews=true')
time.sleep(1)
driver.find_element_by_xpath('//div[@class="MocG8c UFSXYb LMgvRb KKjvXb"]').click()
time.sleep(1)
try:
    driver.find_element_by_xpath('//div[@jsname="V68bde"]//span[@class="vRMGwf oJeWuf"]').click()
except:
    pass

In [ ]:
加載更多留言
透過觀察的方式可以發現，當網頁滾動到最底部的時候會自動加載更多留言，但是每加載5次會需要點擊一次「顯示更多內容」
另外需要留意第一次與第二次之後的「顯示更多內容」是不同的按鈕，所以在這裡使用了一個 try 函數
在這裡我設定做 10個點擊「顯示更多內容」，實際上可以依據資料量的大小自行增加或減少

In [4]:
for i in range(10):
    for i in range(5):
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        driver.find_element_by_xpath('//div[@class="U26fgb O0WRkf oG5Srb C0oVfc n9lfJ M9Bg4d"]').click()
    except:
        try:
            driver.find_element_by_xpath('//div[@class="U26fgb O0WRkf oG5Srb C0oVfc n9lfJ"]').click()
        except:
            pass

In [ ]:
檢視完整留言
對於長度較長的留言需要另外點擊「完整留言」的按鈕才會顯示完整留言，顯示後我們才能抓到資料!
這裡我找的是頁面中有沒有「完整留言」的按鈕可以點擊，有的話就點擊

In [7]:
SeeMore = '//button[@jsname="gxjVle"]'
driver.execute_script("window.scrollTo(0, 0);")
for i, element in enumerate(driver.find_elements_by_xpath(SeeMore)):
    if i % 10 == 0:
        print(i)
    element.click()
    #print(n)

0


ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=84.0.4147.89)


In [ ]:
解析資料
當我們把所有資料加載完成後，我們的資料是html的半結構化的網頁資料
因此我們在這裡需要透過BeautifulSoup來幫我們把資料轉成DataFrame

In [ ]:
soup = BeautifulSoup(driver.page_source)
driver.close()

In [ ]:
list_of_comments = []
for i, element in enumerate(soup.find_all('div',{'jscontroller':'H6eOGe'})):
    if i % 10 == 0:
        print(i)
    try:
        likecounts = element.find('div',{'class':'jUL89d y92BAb'}).text
    except:
        likecounts = str(0)
    
    Content = element.find('span',{'jsname':'bN97Pc'}).text
    # 如果「完整評論」裡面有內容就抓裡面的內容
    if len(element.find('span',{'jsname':'fbQN7e'}).text) >=20:
        Content = element.find('span',{'jsname':'fbQN7e'}).text
    ndf = pd.DataFrame([{'Name':element.find('span', {'class':'X43Kjb'}).text,
                         'Sentimen':element.find('div', {'class':'pf5lIe'}).find('div')['aria-label'],
                         'likecounts':likecounts,
                         'Timestamp':element.find('span',{'class':'p2TkOb'}).text,
                         'Content':Content}],
                        columns = ['Name', 'Timestamp', 'Content', 'Sentimen', 'likecounts'])
    list_of_comments.append(ndf)

In [ ]:
comments = pd.concat(list_of_comments, ignore_index=True)
print(len(comments))
comments.head(20)

In [ ]:
保存資料
先跟 Google Drive 空間串接，再將資料存在 Google Drive 上
資料放在 Google Drive 上的 Colab Notebooks 資料夾中

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
path = '/content/drive/My Drive/Colab Notebooks/'
comments.to_excel(path + 'GooglePlay_Comments.xlsx')